In [ ]:
import cfg
from sys_model import system_model_generator
from cfg import Pen_Container_initializer
from functions import sympy_states_to_func
from functions import jac_func

import sympy as sm
import numpy as np

number_of_pendulums = 1

Pen_Container_initializer(number_of_pendulums)

system_model_generator(cfg.pendata)
fxu= sympy_states_to_func()
dfxu= jac_func(cfg.pendata)

In [ ]:

def constraints(z):

    # n : len(q)
    n = 4
    # number of knotes
    k = 25

    # final time
    h = 2

    # boundry values for x and u
    #x0 = np.array([0, np.pi, 0, 0])
    #xf = np.array([0, 0, 0, 0])
    #u0 = np.array([0])
    #uf = np.array([0])

    #len_z0 = k * (n + 1) - 2 * (n + 1)
    #indx1 = 0
    #indx2 = len_z0 - (k - 2)
    #z = np.block([x0, z0[indx1:indx2], xf, u0, z0[indx2: ], uf])

    x = z[0:n * k].reshape(k, n)
    u = z[n * k: ]
    gz = np.zeros((n * (k - 1), 1))

    for i in range(k - 1):
        x_0 = x[i, :]
        x_1 = x[i + 1, :]
        u_0 = np.array([u[i]])
        u_1 = np.array([u[i + 1]])

        f_0 = fxu(x_0, [u_0])
        f_1 = fxu(x_1, [u_1])

        # computing gradients with this format :
        # df_0= [d/dx_0, d/dx_1, d/du0, d/du1]
        # df_x0u0= dfxu(x_0, u_0)
        # df_0= np.block([df_x0u0[:,0:n],np.zeros((n,n)), df_x0u0[:,-1, np.newaxis] ,  np.zeros((n,1))])
        # df_x1u1= dfxu(x_1, u_1)
        # df_1= np.block([np.zeros((n,n)), df_x1u1[:,0:n], np.zeros((n,1)), df_x0u0[:,-1, np.newaxis]])

        # collocation point and gradient in the same matrix format
        x_c = 0.5 * (x_0 + x_1) + h / 8 * (f_0 - f_1)
        # dx_c = np.block([.5*np.eye(n) , .5*np.eye(n) , np.zeros((n, 2))]) + h/8*(df_0 - df_1)

        # control u_c and gradient in matrix format
        u_c = 0.5 * (u_0 + u_1)
        # du_c= np.array([0 for j in range(2*n)] + [0.5 , 0.5])
        #  xc of the spline and its gradient
        xdot_c = -3 / 2 / h * (x_0 - x_1) - 1 / 4 * (f_0 + f_1)
        # dxdot_c= -3/2/h* np.block([np.eye(n), -1 * np.eye(n), np.zeros((n, 2))]) - 1/4*(df_0 + df_1)
        # calcualte pendulum dynamics at the collocation point and gradient
        f_c = fxu(x_c, u_c)
        # df_c= dfxu(x_c, u_c)
        # add to the gz  vector
        gz[i * n:(i + 1) * n, :] = (xdot_c - f_c)[::, np.newaxis]
        
    print('gz: ', gz)
    
    return np.ravel(gz)

In [ ]:

def objective_func(z):

    k = 25
    n = 4
    h = 2

    #x0 = np.array([0, np.pi, 0, 0])
    xf = z[len(z)-n:]=np.array([0,0,0,0])
    #u0 = np.array([0])
    #uf = np.array([0])
    #len_z0 = k * (n + 1) - 2 * (n + 1)
    indx1 = 0
    #indx2 = len_z0 - (k - 2)

    #z = np.block([x0, z0[indx1:indx2], xf, u0, z0[indx2 :], uf])

    S = Q = np.eye(4)
    R = 0.01 * np.identity(1)
    Ju = 0.5 * xf.dot(S).dot(xf)
    
    lxut = lambda x, u: 0.5 * x.dot(Q).dot(x) + .5 * u.dot(R).dot(u)
    
    x = z[0:n * k].reshape(k, 4)
    u = z[n * k : ]

    hk = h / (k - 1)

    # iterate over x and u and calculate the objective Ju
    for i in range(k - 1):

        # constructing x_k and x_k+1 and integrate l(x,u,t) over
        #  the time period between thease two points using sympson quadrature!
        x_0 = x[i, :]
        x_1 = x[i + 1, :]
        u_0 = np.array([u[i]])
        u_1 = np.array([u[i + 1]])

        f_0 = fxu(x_0, [u_0])
        f_1 = fxu(x_1, [u_1])

        x_c = 0.5 * (x_0 + x_1) + h / 8 * (f_0 - f_1)
        u_c = 0.5 * (u_0 + u_1)

        Ju += hk / 6.0 * (lxut(x_0, u_0) + 4 * lxut(x_c, u_c) + lxut(x_1, u_1))
        print('Ju', Ju)

    return Ju


In [ ]:
from scipy.optimize import minimize
k=25
n=4

collocation_cons = {'type': 'eq', 'fun': constraints}
boundry_cons1={'type':'eq', 'fun': lambda z : z[:4]-np.array([0,np.pi, 0, 0])}
boundry_cons2={'type':'eq', 'fun': lambda z : z[len(z)-4:]-np.array([0, 0, 0, 0])}
boundry_cons3={'type':'eq', 'fun': lambda z : z[k*n]-np.array([0])}
#boundry_cons4={'type':'eq', 'fun': lambda z : z[-1]-np.array([0])}
cons= (collocation_cons, boundry_cons1,boundry_cons2, boundry_cons3)
#cons= (collocation_cons, boundry_cons1,boundry_cons2, boundry_cons3, boundry_cons4 )
z0 = np.array([0., np.pi, 0., 0.] +[1. for i in range(k*n -8)]+ [0., 0., 0., 0.] + [0. for i in range(k)])
res = minimize(objective_func, z0, method='SLSQP', constraints=cons, options={'ftol': 0.7,'disp': True})
#res = minimize(objective_func, z0, method='trust-constr', constraints=cons, options={'xtol': 0.07,'gtol':0.07, 'barrier_tol':0.07, 'disp': True})

